In [12]:
from pymongo import MongoClient
from collections import defaultdict
import datetime
import networkx as nx
import itertools
import matplotlib.pyplot as plt
from pylab import rcParams
import numpy as np

In [13]:
cluster = MongoClient("mongodb://localhost:27017")

db = cluster["smartshark"]

In [14]:
projectCollections = db["project"]

In [15]:
project_kafka = projectCollections.find_one({"name" : "kafka"})
print(project_kafka)

{'_id': ObjectId('5b0fa95e0da3ef431af80833'), 'name': 'kafka'}


In [16]:
pull_request_system_Collection = db['pull_request_system']

In [17]:
pullRequestSystemKafka = pull_request_system_Collection.find_one({"project_id" : project_kafka['_id']})
print(pullRequestSystemKafka)

{'_id': ObjectId('5ff34494cef070edcc19bb6d'), 'project_id': ObjectId('5b0fa95e0da3ef431af80833'), 'url': 'https://api.github.com/repos/apache/kafka/pulls', 'last_updated': datetime.datetime(2021, 1, 6, 19, 51, 16, 785000)}


In [18]:
pull_request_Collection = db['pull_request']

In [19]:
developersWithPRcount = pull_request_Collection.aggregate([
    {"$match" : {"pull_request_system_id" : pullRequestSystemKafka['_id']}},
    {"$group" : {"_id":"$creator_id", "PRCount":{"$sum":1}}}
])

In [20]:
developersWithPRcountList = []
for entry in developersWithPRcount:
    developersWithPRcountList.append([entry['_id'], entry['PRCount']])
    # print(entry['_id'], " -> ", entry['count'])

developersWithPRcountList.sort(key=lambda x : x[1], reverse=True)
totalPRInKafka = 0
for developer in developersWithPRcountList:
    totalPRInKafka += int(developer[1])
    print(developer)

# print("total Pull Requests :", totalPRInKafka)

[ObjectId('58c118ea02ca40f8bfb1f2fc'), 605]
[ObjectId('5b0fc400065f3902048de36a'), 555]
[ObjectId('5b0fc400065f3902098de369'), 488]
[ObjectId('5b0fc400065f39020b8de36a'), 432]
[ObjectId('5bbf118757674ee167468e00'), 350]
[ObjectId('5ff34615c26a57681ea258d1'), 247]
[ObjectId('5b0fc400065f3902028de380'), 226]
[ObjectId('5b0fc420065f3902098e047b'), 219]
[ObjectId('5b0fc401065f3902068de3cf'), 216]
[ObjectId('5ff34647c26a57681ea27204'), 186]
[ObjectId('5b0fc400065f39020e8de371'), 170]
[ObjectId('5ff345ffc26a57681ea252bd'), 170]
[ObjectId('5b0fc421065f39020f8e0180'), 158]
[ObjectId('5b0fc400065f3902058de36a'), 147]
[ObjectId('5ff362c7c26a57681eaa070b'), 133]
[ObjectId('5ff37139c26a57681eae1a66'), 127]
[ObjectId('5b0fc421065f39020b8e0613'), 120]
[ObjectId('5ff35b41c26a57681ea844ee'), 112]
[ObjectId('5bc984bf57674ee167eb67e7'), 104]
[ObjectId('5ff3461ec26a57681ea25da1'), 100]
[ObjectId('5d92f56b3dfe5ddb856b26ea'), 99]
[ObjectId('5db7ffbe3dfe5ddb852b32da'), 90]
[ObjectId('5ff34751c26a57681ea2e97

In [21]:
developerRankDict = defaultdict(float)

for devId, pr_count in developersWithPRcountList:
    developerRankDict[devId] = pr_count / totalPRInKafka


for entry in developerRankDict.items():
    print(entry)

(ObjectId('58c118ea02ca40f8bfb1f2fc'), 0.06152750940709854)
(ObjectId('5b0fc400065f3902048de36a'), 0.05644259127428049)
(ObjectId('5b0fc400065f3902098de369'), 0.04962880097630428)
(ObjectId('5b0fc400065f39020b8de36a'), 0.043933692667548055)
(ObjectId('5bbf118757674ee167468e00'), 0.03559442692972643)
(ObjectId('5ff34615c26a57681ea258d1'), 0.025119495576121226)
(ObjectId('5b0fc400065f3902028de380'), 0.02298382996033764)
(ObjectId('5b0fc420065f3902098e047b'), 0.02227194142174311)
(ObjectId('5b0fc401065f3902068de3cf'), 0.021966846333774027)
(ObjectId('5ff34647c26a57681ea27204'), 0.01891589545408319)
(ObjectId('5b0fc400065f39020e8de371'), 0.01728872165158141)
(ObjectId('5ff345ffc26a57681ea252bd'), 0.01728872165158141)
(ObjectId('5b0fc421065f39020f8e0180'), 0.016068341299705073)
(ObjectId('5b0fc400065f3902058de36a'), 0.014949659310485102)
(ObjectId('5ff362c7c26a57681eaa070b'), 0.013525882233296043)
(ObjectId('5ff37139c26a57681eae1a66'), 0.012915692057357877)
(ObjectId('5b0fc421065f39020b8e06